In [ ]:
from PIL import Image
import os

def create_frame(image_paths, output_path='framed_output.jpg', image_size=(200, 200), spacing=10):
    if len(image_paths) != 8:
        raise ValueError("You must provide exactly 8 image paths.")

    # Resize all images to the same size
    images = [Image.open(path).resize(image_size) for path in image_paths]

    # Calculate the size of the final image
    img_width, img_height = image_size
    frame_width = img_width * 2 + spacing * 3  # 2 columns + 3 spacings
    frame_height = img_height * 4 + spacing * 5  # 4 rows + 5 spacings

    # Create a blank white image
    frame = Image.new('RGB', (frame_width, frame_height), color='white')

    # Paste the left 4 images
    for i in range(4):
        x = spacing
        y = spacing + i * (img_height + spacing)
        frame.paste(images[i], (x, y))

    # Paste the right 4 images
    for i in range(4, 8):
        x = spacing * 2 + img_width
        y = spacing + (i - 4) * (img_height + spacing)
        frame.paste(images[i], (x, y))

    # Save the final frame
    frame.save(output_path)
    print(f"Frame saved to {output_path}")

# Example usage:
# Provide 8 valid image paths here
image_paths = [
    "ChildNet\Train\Angry\2f17de29a2_face1.jpg", 
    "ChildNet\Train\Happiness\2c7dc5768c_face1.jpg", 
    "ChildNet\Train\Sad\3c1df9cc28_face1.jpg", 
    "ChildNet\Train\Surprised\9b1c9adfc4_face1.jpg",
    "TIF_DF\Angry\A08M12-JTPC-5437AN-SA.jpg", 
    "TIF_DF\Happiness\A05F7-JTP-4929HA.jpg", 
    "image7.jpg", 
    "image8.jpg"
]

create_frame(image_paths)

In [4]:
from PIL import Image, ImageDraw, ImageFont
import os

def create_image_grid_with_captions(image_caption_list, output_path='grid_output.jpg',
                                    image_size=(200, 200), spacing=20, caption_height=30, font_path='arial.ttf'):
    if len(image_caption_list) != 14:
        raise ValueError("You must provide exactly 14 (image_path, caption) pairs.")

    images = []
    captions = []

    for img_path, caption in image_caption_list:
        img = Image.open(img_path).resize(image_size)
        images.append(img)
        captions.append(caption)

    # Grid dimensions
    num_columns = 7
    num_rows = 2
    img_width, img_height = image_size

    # Total image + caption height
    tile_height = img_height + caption_height

    # Final image size
    total_width = num_columns * img_width + (num_columns + 1) * spacing
    total_height = num_rows * tile_height + (num_rows + 1) * spacing

    # Create new image
    result = Image.new('RGB', (total_width, total_height), 'white')
    draw = ImageDraw.Draw(result)

    # Load font
    try:
        font = ImageFont.truetype(font_path, 18)
    except IOError:
        font = ImageFont.load_default()
        print("Custom font not found. Using default font.")

    # Paste each image and caption
    for idx, (img, caption) in enumerate(zip(images, captions)):
        row = idx // num_columns
        col = idx % num_columns

        x = spacing + col * (img_width + spacing)
        y = spacing + row * (tile_height + spacing)

        result.paste(img, (x, y))

        # Caption center-aligned under image
        bbox = draw.textbbox((0, 0), caption, font=font)
        text_width = bbox[2] - bbox[0]
        text_x = x + (img_width - text_width) // 2
        text_y = y + img_height + 5
        draw.text((text_x, text_y), caption, fill='black', font=font)

    # Save final image
    result.save(output_path)
    print(f"Grid saved to {output_path}")

# Example image/caption list
image_caption_list = [
    (r"ChildNet\Train\Angry\2f17de29a2_face1.jpg", "Angry"),
    (r"ChildNet\Train\Disgust\79b25715e8_face1.jpg", "Disgust"),
    (r"ChildNet\Train\Happiness\2c7dc5768c_face1.jpg", "Happy"),
    (r"ChildNet\Train\Fear\989e70e327_face1.jpg", "Fear"),
    (r"ChildNet\Train\Neutral\1fce088efa_face1.jpg", "Neutral"),
    (r"ChildNet\Train\Sad\3c1df9cc28_face1.jpg", "Sad"),
    (r"ChildNet\Train\Surprised\9b1c9adfc4_face1.jpg", "Surprise"),

    (r"TIF_DF\Angry\A08M12-JTPC-5437AN-SA.jpg", "Angry"),
    (r"TIF_DF\Disgust\A07M4-NONE-5320DI.jpg", "Disgust"),
    (r"TIF_DF\Fear\A16F6-JTPC-7340AF.jpg", "Fear"),
    (r"TIF_DF\Happiness\A05F7-JTP-4929HA.jpg", "Happy"),
    (r"TIF_DF\Neutral\A06M5-JTPC-5083NE.jpg", "Neutral"),
    (r"TIF_DF\Sad\A06M5-JTPC-5148SA.jpg", "Sad"),
    (r"TIF_DF\Surprised\A04M6-JTPC-4783SU.jpg", "Surprise"),
]

# Run the function
create_image_grid_with_captions(image_caption_list, font_path="arial.ttf")

Grid saved to grid_output.jpg


In [6]:
from PIL import Image, ImageDraw, ImageFont
import os

def create_dual_frame(image_caption_list, output_path='dual_frame.jpg',
                      image_size=(200, 200), spacing=20, caption_height=30,
                      title_height=50, font_path='arial.ttf'):
    if len(image_caption_list) != 14:
        raise ValueError("You must provide exactly 14 (image_path, caption) pairs.")

    # Split into two datasets
    dataset1 = image_caption_list[:7]
    dataset2 = image_caption_list[7:]

    num_columns = 7
    img_width, img_height = image_size
    tile_height = img_height + caption_height
    total_width = num_columns * img_width + (num_columns + 1) * spacing
    single_frame_height = title_height + tile_height + spacing * 2
    total_height = single_frame_height * 2  # Two rows (ChildNet, TIF_DF)

    result = Image.new('RGB', (total_width, total_height), 'white')
    draw = ImageDraw.Draw(result)

    # Load font
    try:
        font = ImageFont.truetype(font_path, 18)
        title_font = ImageFont.truetype(font_path, 28)
    except IOError:
        font = ImageFont.load_default()
        title_font = ImageFont.load_default()
        print("Custom font not found. Using default font.")

    def draw_frame(start_y, dataset, title):
        # Draw title centered
        title_bbox = draw.textbbox((0, 0), title, font=title_font)
        title_width = title_bbox[2] - title_bbox[0]
        title_x = (total_width - title_width) // 2
        draw.text((title_x, start_y), title, fill='black', font=title_font)

        y_offset = start_y + title_height
        for idx, (img_path, label) in enumerate(dataset):
            try:
                img = Image.open(img_path).resize(image_size)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue

            x = spacing + idx * (img_width + spacing)
            result.paste(img, (x, y_offset))

            # Caption
            caption_bbox = draw.textbbox((0, 0), label, font=font)
            text_width = caption_bbox[2] - caption_bbox[0]
            text_x = x + (img_width - text_width) // 2
            text_y = y_offset + img_height + 5
            draw.text((text_x, text_y), label, fill='black', font=font)

    # Draw both frames
    draw_frame(0, dataset1, "ChildNet")
    draw_frame(single_frame_height, dataset2, "TIF_DF")

    result.save(output_path)
    print(f"Dual frame saved to {output_path}")

# Example usage
image_caption_list = [
    (r"ChildNet\Train\Angry\2f17de29a2_face1.jpg", "Angry"),
    (r"ChildNet\Train\Disgust\79b25715e8_face1.jpg", "Disgust"),
    (r"ChildNet\Train\Happiness\2c7dc5768c_face1.jpg", "Happy"),
    (r"ChildNet\Train\Fear\989e70e327_face1.jpg", "Fear"),
    (r"ChildNet\Train\Neutral\1fce088efa_face1.jpg", "Neutral"),
    (r"ChildNet\Train\Sad\3c1df9cc28_face1.jpg", "Sad"),
    (r"ChildNet\Train\Surprised\9b1c9adfc4_face1.jpg", "Surprise"),

    (r"TIF_DF\Angry\A08M12-JTPC-5437AN-SA.jpg", "Angry"),
    (r"TIF_DF\Disgust\A07M4-NONE-5320DI.jpg", "Disgust"),
    (r"TIF_DF\Fear\A16F6-JTPC-7340AF.jpg", "Fear"),
    (r"TIF_DF\Happiness\A05F7-JTP-4929HA.jpg", "Happy"),
    (r"TIF_DF\Neutral\A06M5-JTPC-5083NE.jpg", "Neutral"),
    (r"TIF_DF\Sad\A06M5-JTPC-5148SA.jpg", "Sad"),
    (r"TIF_DF\Surprised\A04M6-JTPC-4783SU.jpg", "Surprise"),
]

# Run
create_dual_frame(image_caption_list, font_path="arial.ttf")

Dual frame saved to dual_frame.jpg
